In [26]:
import pandas as pd
import os

In [27]:
df = pd.DataFrame()
dir = "your-directory"
files = os.listdir(dir)
for f in files:
    if not f.endswith("csv"):
        continue
    if df.empty:
        df = pd.read_csv(dir+f)
    else:
        df = pd.concat([df,pd.read_csv(dir+f)],axis=0)
df.drop(["Issue Code","ISIN Code"],axis=1,inplace=True)
df['Issue Date (datetime)'] = pd.to_datetime(df['Issue Date'])
df.sort_values(by ="Issue Date (datetime)",inplace=True)
df.drop("Issue Date (datetime)",axis = 1, inplace=True)
df.drop_duplicates(keep='first',inplace=True)

In [29]:
def check_growth(df):
  for row in df.T.columns[4:]:
    if "Date" in row:
      continue
    series = df.loc[row].astype(float)
    change = [0]
    for i in range(1,len(series)):
        change.append((series[i] - series[i-1]) / series[i-1])
    df.loc[f"{row} Growth"] = change
  return df

In [30]:
df_6_months = df[df['Auction Tenor (Month)'] == 6]
df_6_months.reset_index(drop=True,inplace=True)
df_6_months = df_6_months.T
df_6_months = check_growth(df_6_months)
df_6_months.drop("Auction Tenor (Month)",axis = 0,inplace=True)

In [31]:
final_df = pd.DataFrame()
for feature in df_6_months.T.columns:
    if "Date" in feature: # Not predicting and just appending to the final df.
      temp = df_6_months.loc[feature][-3:].values.flatten()
      final_df[f"{feature}1"] = pd.Series(temp[0])
      final_df[f"{feature}2"] = pd.Series(temp[1])
      final_df[f"{feature}3"] = pd.Series(temp[2])
    else:
      growth_list = df_6_months.T.columns.to_list()
      target = feature #'Cut-off Yield (%)'
      # Window size for features (3 periods)
      window_size = 3
      # Extract features and target variables
      X, y = [], []
      for i in range(window_size, len(df_6_months.columns),window_size): # Overwrapping
          X.append(df_6_months.drop([target],axis = 0).iloc[:,i-window_size:i].values.flatten())  # Features (3 periods of data)
          y.append(df_6_months.T[target][i])  # Target for the fourth period

      three_period_X_cols = []
      feature_list_copy = growth_list.copy()
      feature_list_copy.remove(target)
      for col in feature_list_copy:
        three_period_X_cols.append(f"{col}1")
        three_period_X_cols.append(f"{col}2")
        three_period_X_cols.append(f"{col}3")
      train_temp = pd.DataFrame(data = X_train, columns = three_period_X_cols)
      train_temp[target] = y_train
      test_temp = pd.DataFrame(data = X_test, columns = three_period_X_cols)
      test_temp[target] = y_test
      train_df = train_temp
      test_df = test_temp
      train_df = train_df.reset_index(drop = True)
      test_df = test_df.reset_index(drop = True)
      from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error
      # Define and train XGBRegressor model
      from xgboost import XGBRegressor
      model = XGBRegressor(max_depth = 5, n_estimators=100)

      # training
      X_train = train_df.copy().iloc[:,9:].drop(target,axis=1)
      y_train = train_df[target]
      model.fit(X_train,y_train)

      total_precision = 0
      print("Test results")
      X_test = test_df.copy().iloc[:,9:].drop(target,axis=1)
      y_test = test_df[target]
      # Evaluate and predict
      predictions = model.predict(X_test)
      pct_error = mean_absolute_percentage_error(y_test,predictions)
      mse = mean_squared_error(y_test,predictions)
      imp_df = pd.DataFrame()
      imp_df['Feature'] = model.feature_names_in_
      imp_df['Importance']= model.feature_importances_
      imp_df.sort_values(by='Importance', ascending=False,inplace=True)
      print(f"{target} Top 3 Important Features: {imp_df[:3]['Feature'].to_list()}")
      print(f"{target} MSE: {mse}")
      print(f"{target} Prediction Percentage Error: {pct_error}")
      curr_X = []
      curr_X.append(df_6_months.drop([target],axis = 0).iloc[:,-3:].values.flatten())  # Features (3 periods of data)
      curr_df = pd.DataFrame(data = curr_X, columns = three_period_X_cols)
      predictions = model.predict(curr_df.iloc[:,9:])
      final_df[f'{target} Prediction'] = predictions

Test results
Auction Amount (S$M) Top 3 Important Features: ['Average Yield (%)1', 'Cut-off Yield (%)1', 'Auction Amount (S$M) Growth1']
Auction Amount (S$M) MSE: 3845641.6141334176
Auction Amount (S$M) Prediction Percentage Error: 0.2965570144338968
Test results
Amount Applied (S$M) Top 3 Important Features: ['Auction Amount (S$M)3', 'Median Yield (%)3', 'Cut-off Yield (%)1']
Amount Applied (S$M) MSE: 12732782.323326364
Amount Applied (S$M) Prediction Percentage Error: 0.2184851708783375
Test results
Bid-to-Cover Ratio Top 3 Important Features: ['Amount Applied (S$M)2', 'Amount Applied (S$M) Growth1', 'Average Yield (%)3']
Bid-to-Cover Ratio MSE: 0.1015147799640825
Bid-to-Cover Ratio Prediction Percentage Error: 0.12769530452697886
Test results
Cut-off Yield (%) Top 3 Important Features: ['Cut-off Price1', 'Auction Amount (S$M) Growth1', 'Auction Amount (S$M)2']
Cut-off Yield (%) MSE: 0.05453164969352495
Cut-off Yield (%) Prediction Percentage Error: 0.05244652615850036
Test results
C

In [33]:
last_exercise_df = pd.DataFrame(df[df['Auction Tenor (Month)'] == 6].iloc[-1]).T
last_exercise_df['Auction Date3'] = last_exercise_df['Auction Date']
last_exercise_df['Issue Date3'] = last_exercise_df['Issue Date']
last_exercise_df['Maturity Date3'] = last_exercise_df['Maturity Date']
for col in final_df.columns:
    if col not in last_exercise_df.columns:
        if "Date" in col:
            last_exercise_df[col] = ["-"]
        if "Growth" in col:
            last_exercise_df[col] = last_exercise_df[col.replace(" Growth Prediction","")]
        else:
            last_exercise_df[col] = last_exercise_df[col.replace(" Prediction","")]
last_exercise_df = last_exercise_df[final_df.columns]

final_df = pd.concat([final_df,last_exercise_df],axis=0)

In [34]:
final_df

,Auction Date1,Auction Date2,Auction Date3,Issue Date1,Issue Date2,Issue Date3,Maturity Date1,Maturity Date2,Maturity Date3,Auction Amount (S$M) Prediction,...,Average Price Prediction,Auction Amount (S$M) Growth Prediction,Amount Applied (S$M) Growth Prediction,Bid-to-Cover Ratio Growth Prediction,Cut-off Yield (%) Growth Prediction,Cut-off Price Growth Prediction,Median Yield (%) Growth Prediction,Median Price Growth Prediction,Average Yield (%) Growth Prediction,Average Price Growth Prediction
0,23 May 2024,06 Jun 2024,20 Jun 2024,28 May 2024,11 Jun 2024,25 Jun 2024,26 Nov 2024,10 Dec 2024,24 Dec 2024,4562.686523,...,98.387207,0.014966,-0.007812,-0.029976,0.009499,-0.000029,0.002099,-0.000296,0.12724,-0.000392
0,-,-,20 Jun 2024,-,-,25 Jun 2024,-,-,24 Dec 2024,6600,...,98.464,6600,15457.038,2.34,3.74,98.135,3.54,98.235,3.08,98.464


In [36]:
import datetime
date = (datetime.date.today()+datetime.timedelta(days=14)).strftime("%Y-%m-%d")
final_df.to_excel(f"{dir}T-Bill Prediction {date}.xlsx",index=False)


In [80]:
import openai
import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")

# Replace 'your-api-key' with your actual API key
openai.api_key = 'your-api-key'

json_sample = {
    "Cut-off Yield Growth" : "Your prediction",
}

def get_response_from_chatgpt(prompt):
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages = [
        {"role": "system", "content": f"""You are an expert at analysing and deducing information.
        Only use the information provided in forming your judgements.
        Return the rates of each feature like in the format of the {json_sample}.
        """},
        {"role": "user", "content": prompt},
        ],
    )
    return response.choices[0].message.content

def prompter_gpt_conditions(data):
    """This function will provide a financial expert's response towards user's questions."""
    prompt = f"""
    This data contains prior results. The data is {data}. Account for financial conditions as of today in your analysis and predictions.
    Calculate the expected rates going for the next period and explain your rationale.
    Return it in the form of the {json_sample} in JSON. 
    Don't return any code.
    """
    if max_tokens != "":
        max_tokens = int(max_tokens)
        response = get_response_from_chatgpt(prompt + f"Keep your answer within {str(max_tokens)} tokens", max_tokens = max_tokens)
    else:
        response = get_response_from_chatgpt(prompt)
    print(f"Number of tokens used: {len(encoding.encode(prompt))}")
    tokens=len(encoding.encode(prompt))
    return response,tokens

def prompter_gemini_conditions(data, conditions, max_tokens=4000):
    """This function will provide a financial expert's response towards user's questions."""
    prompt = f"""
    This data contains prior results. The data is {data}. Financial conditions today are as follows: {conditions}
    Calculate the expected rates going for the next period and explain your rationale.
    Return it in the form of the {json_sample} in JSON. 
    Don't return any code.
    """
    if max_tokens != "":
        max_tokens = int(max_tokens)
        response = get_response_from_chatgpt(prompt + f"Keep your answer within {str(max_tokens)} tokens", max_tokens = max_tokens)
    else:
        response = get_response_from_chatgpt(prompt)
    print(f"Number of tokens used: {len(encoding.encode(prompt))}")
    tokens=len(encoding.encode(prompt))
    return response,tokens



In [81]:
# Ask for today's financial conditions from Gemini
gemini_conditions = """
Here's a summary of the global financial condition based on publicly available information as of today, June 20, 2024:

Mixed Signals:

Global Growth Slowing: The International Monetary Fund (IMF) has revised down global growth projections for 2022 and 2023 to 3.6%, reflecting ongoing challenges.
Uneven Recovery: Different regions are experiencing varying recovery paces. India, for example, reported a GDP growth rate of 8.2% for the fiscal year 2023-24, which is positive news.
Looser Financial Conditions: In the United States, the Federal Reserve Bank of Chicago's National Financial Conditions Index suggests looser financial conditions as of June 7th, 2024.
Uncertainties:

Geopolitical Tensions: Ongoing geopolitical tensions continue to pose risks to the global economy.
Inflation: Inflation remains a concern in many countries, although the specific situation may vary.
Interest Rate Hikes: Central banks in some countries, like the US Federal Reserve, are considering raising interest rates to combat inflation, which could impact financial markets.
Overall, the global financial condition presents a mixed picture. While there are signs of recovery in some areas, there are also significant uncertainties that could affect future performance.

"""

In [82]:
response,tokens = prompter_gpt_conditions(df_6_months)

Number of tokens used: 3438


In [83]:
import json

In [84]:
reversed_response = response[::-1]
backmarker = len(response) - reversed_response.find("}")
cleaned_response = response[response.find("{"):backmarker+1]
cleaned_response = cleaned_response.replace("\n", "")
cleaned_response = cleaned_response.replace("  ","")
responses = []
while len(cleaned_response)>0:
    start = cleaned_response.find("{")
    end = cleaned_response.find("}")
    trimmed_response = cleaned_response[start:end+1]
    responses.append(trimmed_response)
    cleaned_response = cleaned_response[end+1:]
data = [json.loads(string) for string in responses]
gpt_df = pd.DataFrame(data).astype(str)

In [85]:
response,tokens = prompter_gemini_conditions(df_6_months,gemini_conditions)
reversed_response = response[::-1]
backmarker = len(response) - reversed_response.find("}")
cleaned_response = response[response.find("{"):backmarker+1]
cleaned_response = cleaned_response.replace("\n", "")
cleaned_response = cleaned_response.replace("  ","")
responses = []
while len(cleaned_response)>0:
    start = cleaned_response.find("{")
    end = cleaned_response.find("}")
    trimmed_response = cleaned_response[start:end+1]
    responses.append(trimmed_response)
    cleaned_response = cleaned_response[end+1:]
data = [json.loads(string) for string in responses]
gemini_df = pd.DataFrame(data).astype(str)
combined_df = pd.concat([gpt_df,gemini_df],axis=0)

Number of tokens used: 3686


In [86]:
columns = combined_df.columns
columns = columns.insert(0,"Predictor")
combined_df['Predictor'] = ['GPT',"Gemini"]
combined_df = combined_df[columns]
combined_df.to_excel(f"{dir} T-Bill GPT Prediction {date}.xlsx",index=False)